# Project 4 - Part II

# C L E A N I N G

<div class="alert alert-block alert-info">
<b>Import all required libraries and folders</b> 
</div>

In [1]:
# Pandas and Numpy
import pandas as pd 
import numpy as np

# Regular Expression
import re

<div class="alert alert-block alert-info">
<b>Read Seek scrapped data into a DataFrame</b> 
</div>

In [2]:
SEEK_Data = pd.read_csv('Datasets/SEEK_Scrapped_Data.csv',index_col=0)
#SEEK_Data_Other = pandas.read_csv('Datasets/SEEK_Scrapped_Data_Other.csv',index_col=0)

<div class="alert alert-block alert-info">
<b>Concatinate if multiple files and print the first 5 records</b> 
</div>

In [3]:
df = pd.concat([SEEK_Data]).drop_duplicates(['description']).reset_index(drop=True)
#df = pandas.concat([SEEK_Data,SEEK_Data_Other]).drop_duplicates(['description']).reset_index(drop=True)

df.head()

,title,description,advertiser,date,salary,worktype,classificaiton,sub_classificaiton,city,suburb,link,category
0,Web Analytics Manager,NaN,this company,13 Feb 2019,NaN,Full Time,Marketing & Communications,Marketing & Communications,Melbourne,Melbourne,https://www.seek.com.au/job/38337813?type=prom...,Data-Science
1,Data Science Consultant,\r\n\r\n\r\nGreat location in Sydney CBD - Mod...,this company,15 Feb 2019,NaN,Full Time,Information & Communication Technology,Information & Communication Technology,Sydney,Sydney,https://www.seek.com.au/job/38363396?type=stan...,Data-Science
2,Data Scientist - Sponsorship Available,\r\nAbout the business and the role\r\n \r\nMy...,this company,8 Feb 2019,NaN,Full Time,Science & Technology,Science & Technology,Sydney,Sydney,https://www.seek.com.au/job/38305983?type=stan...,Data-Science
3,Head of Data Science,\r\nAbout our Client: We are currently workin...,this company,16 Feb 2019,"£200000.00 - £300k p.a. + super, bonus and wid...",Full Time,Consulting & Strategy,Consulting & Strategy,Sydney,Sydney,https://www.seek.com.au/job/38366878?type=stan...,Data-Science
4,Senior Data Scientist,\r\n\r\nWork with a global leading enterprise\...,this company,15 Feb 2019,NaN,Full Time,Information & Communication Technology,Information & Communication Technology,Sydney,Sydney,https://www.seek.com.au/job/38363660?type=stan...,Data-Science


In [4]:
df.shape

(4196, 12)

In [5]:
df.isnull().sum()

title                    0
description              1
advertiser              20
date                     0
salary                2666
worktype                 0
classificaiton           0
sub_classificaiton       0
city                     0
suburb                   0
link                     0
category                 0
dtype: int64

<div class="alert alert-block alert-info">
<b>Quick SALARY Cleaning</b> 
<br>Use Regular Expression
<br>Strip on blank and print the first 5 records
</div>

In [6]:
# Using replace and regex to clean the salary column

df[['salary']] = df[['salary']].replace('[Kk]\b', '000', regex=True)
df[['salary']] = df[['salary']].replace('\.[0-9][0-9] ', ' ', regex=True)
df[['salary']] = df[['salary']].replace('[!a-zA-Z+&,.\$£/:)(]', '', regex=True)
df[['salary']] = df[['salary']].replace('....%', '', regex=True)

df.salary.str.strip().head()

0             NaN
1             NaN
2             NaN
3    200000 - 300
4             NaN
Name: salary, dtype: object

In [7]:
def match_salary(x):
    try:
        find = re.compile('\d+')
        return find.findall(x)
    except:
        return []
    
df.salary = df.salary.apply(match_salary)

df.salary.head()

0               []
1               []
2               []
3    [200000, 300]
4               []
Name: salary, dtype: object

<div class="alert alert-block alert-info">
<b>Convert Salary to annual salary</b> 
</div>

In [8]:
def final_salary(x):
    
    def correct_salary(y):
        if y == 0:
            return np.nan
        elif y <= 50:
            return y * 8 * 250
        elif y < 400 and y > 50:
            return y * 1000
        elif y < 2000 and y >= 500: 
            return y * 250
        else:
            return y
    
    x = [correct_salary(int(e)) for e in x]    
    return  np.nan if len(x) == 0 else np.mean(x)

df.salary = df.salary.map(lambda x: final_salary(x))

df.salary = df.salary.round(0)

df.salary.head()

0         NaN
1         NaN
2         NaN
3    250000.0
4         NaN
Name: salary, dtype: float64

<div class="alert alert-block alert-info">
<b>NULL any salary less than 30K</b> 
</div>

In [9]:
print (df.loc[df.salary < 30000,'salary'])

df.loc[df.salary < 30000,'salary'] = np.nan

371       408.0
437     27082.0
514       425.0
631     27373.0
638     27373.0
640     27373.0
641     27373.0
667      7665.0
957     18000.0
962       466.0
1070      462.0
1086    12000.0
1206      425.0
1327    10000.0
1411    24000.0
1538    12000.0
1764    12000.0
1778    12000.0
2093     2000.0
2190     4250.0
2240    28000.0
2252    27500.0
2309    28000.0
2562      476.0
2633    12000.0
2839    24000.0
2966      425.0
3107      450.0
4030    10000.0
4083    10000.0
4086     4000.0
Name: salary, dtype: float64


In [10]:
df.salary.notnull().sum()

1000

<div class="alert alert-block alert-info">
<b>Clean DESCRIPTION</b> 
</div>

In [11]:
df.description.head()

0                                                  NaN
1    \r\n\r\n\r\nGreat location in Sydney CBD - Mod...
2    \r\nAbout the business and the role\r\n \r\nMy...
3    \r\nAbout our Client:  We are currently workin...
4    \r\n\r\nWork with a global leading enterprise\...
Name: description, dtype: object

In [12]:
df[['description']] = df[['description']].replace(['\\xc2', '\\xa0', '\\xe2', '\\x84',
                                                   '\\xa2', '\\x80', '\\x93', '\\n',   
                                                   '\\r'], ' ', regex=True)

In [13]:
df.description[10]

"  Nielsen is a global performance management company that provides a comprehensive understanding of what consumers Watch and Buy. Nielsen’s Watch segment provides media and advertising clients with Total Audience measurement services across all            devices where content — video, audio, and text — is consumed. The Buy segment offers consumer packaged goods manufacturers and retailers the industry’s only global view of retail performance measurement.By integrating information from            its Watch and Buy segments and other data sources, Nielsen provides its clients with both world-class measurement as well as analytics that help improve performance. Nielsen, an S&P 500 company, has operations in over 100 countries that            cover more than 90 percent of the world’s population. For more information, visit www.nielsen.com     The Opportunity     We are currently seeking a Senior Executive to join our Watch Data Science team. The Data Science organization is at the core o

<div class="alert alert-block alert-info">
<b>Cleaning Done copy DataFrame to a file</b> 
</div>

In [14]:
df.to_csv('Datasets/SEEK_Cleaned.csv')

<div class="alert alert-block alert-danger">
    <b>CODE from Here on is NOT required only for REFERENCE</b>
</div>

<div class="alert alert-block alert-warning">
<b>ORIGINAL Salary Cleaning that took for ever to clean</b>
</div>

In [ ]:
df.salary.unique()

In [ ]:
# part1 = pandas.read_csv('/Users/sergi/Downloads/DSI/Project-4/Datasets/job_df_part1.csv',index_col=0)
# #part2 = pandas.read_csv('Datasets/job_df_part2.csv',index_col=0)
# df = pandas.concat([part1]).drop_duplicates(['description']).reset_index(drop=True)

# df.salary.unique();

In [ ]:
df[['new_salary']] = df[['salary']]
df[['new_salary']] = df[['new_salary']].replace('[+]|plus','~', regex=True)
df[['new_salary']] = df[['new_salary']].replace('[!+&,\/:)(]', '', regex=True)
df[['new_salary']] = df[['new_salary']].replace('K-|k-|K |k |K~|k~|k\'', '000 ', regex=True)
df[['new_salary']] = df[['new_salary']].replace(' to ', ' - ', regex=True)
df[['new_salary']] = df[['new_salary']].replace('[*\t\r\n\-]', ' ', regex=True)
df[['new_salary']] = df[['new_salary']].replace('per annum|p.a|annuation', '', regex=True)
df[['new_salary']] = df[['new_salary']].replace('%', '%%', regex=True)
df[['new_salary']] = df[['new_salary']].replace(np.nan, ' ', regex=True)
df[['new_salary']] = df[['new_salary']].replace('#([\w.;]+)', '', regex=True)
#df[['new_salary']] = df[['new_salary']].replace('\.[0-9][0-9]', '', regex=True)
df[['new_salary']] = df[['new_salary']].replace('[~|~]', '', regex=True)
df[['new_salary']] = df[['new_salary']].replace('per hour|per hr| hr|phr|Hour|hourly|Per ', 'ph', regex=True)
df[['new_salary']] = df[['new_salary']].replace('per day|day |daily|Day', 'pd', regex=True)

df[['N_salary']] = df[['new_salary']].replace(np.nan, ' ', regex=True)
df[['Super']] = df[['salary']]
words = ['~ super', '~ Super', 'including super', '% super', '% Super', ' super ', ' Super ', ' super', ' Super', '%super']
hold_word = []
new_string  = []                                               # Create empty list for store new string when reversing
index = 0
for string in df.new_salary[:]:                                   # Iterate thru each Salary in the file
    new_salary = []
    hold_string = []                                           # Create empty list to append characters
    first_time_flag = 'Y'                                      # Set flag to be First time 
    
    flag_1 = 'Y'
    super_perc = []
    hold_word = ' '
#    print (string)
    for word in words:
        if word in string:
#            print ('found\t', word)
            hold_word = '9.5%'
            string = string.replace(word, '')
#            print ('Removed word\t', string)        
    if '%' in string:
        word_reverse = string[::-1]
#        print ('Percent found', word_reverse)
        for char in word_reverse:
            if char == '%':
                flag_1 = 'N'
            if char == ' ':
                break
            else:
                super_perc.insert(0,(char))
        super_perc = ''.join(super_perc[::])
#        print ('super_perc', super_perc, '\t', flag_1)
    if flag_1 == 'N':
        hold_word = super_perc
        string = string.replace(super_perc, '')
#    print ('A string\t', string, '\thold_word\t', hold_word, '\tPerc\t',super_perc)
    df.Super[index] = hold_word
    for char in string:                                        # iterate thru each character in the string
        if first_time_flag == 'Y':                             # Check if First time 
            if char == '$' or char == '£' or char.isnumeric(): # when the first $,£ or numeric found  
                first_time_flag = 'N'                          # Set flag to not be First time anymore
        if first_time_flag == 'N':                             # If it's not first time meas it found $,£ or numeric
            hold_string.insert(0,(char))                       # append to the new string
            
    if first_time_flag == 'Y':                                 # Rows with no $, £ or numerics, string is all Alpha
        hold_string.insert(0,(' '))                            # Store a space 
        
    hold_string = ''.join(hold_string[::-1])                   # Reverse back the new string
    df.N_salary[index] = ''.join(hold_string[::])              # Reverse back the new string
    index+=1

# In the column 'Salary1', extract single digit in the strings
#df[['Salary1','Salary2']] = df['N_salary'].str.split('^\S*', expand=True)


df.N_salary.unique()

In [ ]:
hold_field_1 = []
hold_field_2 = []
hold_field_3 = []
for salary in df.N_salary.str.split():
    length = (len(salary))
#    print (salary)
    count = 0
    if length > 0:
        while count <= 3 and count < length:
            if count == 0:
                hold_field_1.append(salary[count])
            if count == 1:
                hold_field_2.append(salary[count])
            if count == 2:
                hold_field_3.append(salary[count])
            count +=1
#    print (length)

    if count < 4:
        if count == 0:
            hold_field_1.append(' ')
            hold_field_2.append(' ')
            hold_field_3.append(' ')
        if count == 1:
            hold_field_2.append(' ')
            hold_field_3.append(' ')
        if count == 2:
            hold_field_3.append(' ')
df['Salary1'] = hold_field_1
df['Salary2'] = hold_field_2
df['Salary3'] = hold_field_3

df[['Salary1']] = df[['Salary1']].replace('\.[0-9][0-9]', '', regex=True)
df[['Salary1']] = df[['Salary1']].replace('[!a-zA-Z+&,.\$£%/:)(\']', '', regex=True) 
df[['Salary1']] = df[['Salary1']].replace(' ', np.nan, regex=True)
#df[['Salary1']] = pd.to_numeric(df['Salary1'])
df.Salary1 = pd.to_numeric(df.Salary1, errors='coerce').fillna(0).astype(int)

In [ ]:
df['Currency'] = hold_field_3          # just create a new column to overide
pd_list = ['p.d', 'day', 'pd', 'p.d.', 'Daily', 'day ', 'p.d ', 'p.d. ', 'pd ', 'PD']
ph_list = ['p.h', 'hour', 'ph', 'p.h.', 'Hour', 'p.h.', 'ph ', 'p.h. ', 'hr ']
Currency_list = ['US', '£']

index = 0
for i, row in df.iterrows():
    if df.Salary1[index] <= 20: 
        df.Salary1[index] = np.nan
        
    if '%' in df.Salary2[index]: 
        df.Super[index] = df.Salary2[index]

    for x in pd_list:
        if x in df.Salary2[index]:
            df.Salary3[index] = 'pd'
        if x in df.Salary3[index]:
            df.Salary3[index] = 'pd'
    for y in ph_list:
        if y in df.Salary2[index]: 
            df.Salary3[index] = 'ph'
        if y in df.Salary3[index]: 
            df.Salary3[index] = 'ph'
    for z in Currency_list:
        df.Currency[index] = ' '
        if z in df.Salary2[index]: 
            df.Currency[index] = z
    index +=1

In [ ]:
df[['Salary2']] = df[['Salary2']].replace('\.[0-9][0-9]', '', regex=True)
df[['Salary2']] = df[['Salary2']].replace('[!a-zA-Z+&,.\$£%/:)(\']', '', regex=True) 
df[['Salary2']] = df[['Salary2']].replace(' ', np.nan, regex=True)
df.Salary2 = pd.to_numeric(df.Salary2, errors='coerce').fillna(0).astype(int)
df[['Salary3']] = df[['Salary3']].replace('k|K-|k-|K |k |K~|k~|k\'', '000 ', regex=True)
df[['Salary3']] = df[['Salary3']].replace('[\$£%\']', '', regex=True) 
df[['Super']] = df[['Super']].replace('[!a-zA-Z+&,.\$£%/:)(\']', '', regex=True) 

In [ ]:
index = 0
for i, row in df.iterrows():
#    print (df.Salary3[index])
    if 'pd' in df.Salary3[index]or 'ph' in df.Salary3[index]: 
        continue
    else:
        if (df.Salary3[index]).isnumeric():
            
            if int(df.Salary3[index]) < 50:
                df.Salary3[index] = np.nan
            if int(df.Salary3[index]) < 250:
                df.Salary2[index] = int(df.Salary3[index])
                df.Salary3[index] = 'ph'
            else:
                if int(df.Salary3[index]) > 249 and int(df.Salary3[index]) < 2000:
                    df.Salary2[index] = int(df.Salary3[index])
                    df.Salary3[index] = 'pd'
                else:
                    df.Salary2[index] = int(df.Salary3[index])
                    df.Salary3[index] = ' '
        else:   
            df.Salary3[index] = ' '
#         else:
#             df.Salary3[index] = ' '
                    
#    print (df.Salary3[index])
        
    index +=1 
df[['Salary3']] = df[['Salary3']].replace('[!a-zA-Z+&,.\$£%/:)(\']', '', regex=True) 

In [ ]:
df.to_csv('Datasets/job_df_clean.csv', encoding='utf8')
df.dtypes

In [ ]:
df.Salary3.unique()

<div class="alert alert-block alert-success">
<b>Code NOT Used which can HELP ONLY</b>
</div>

In [ ]:
for i in df.Salary1:
    print (i)
    x = int(i)
    if i.is_number():
        print (i)

In [ ]:
pd.to_numeric(df.Salary1)

In [ ]:
index = 0
for key,value in df.iteritems():
    print (index)
    print (df.Salary1[index])
    index +=1
    
print ('...',index)
#    print (df.Salary1)
#    print (key)

In [ ]:
index = 0
for i, row in df.iterrows():
    if df.Salary1[index] <= 20: 
        print ('Before\t', df.Salary1[index])
        df.Salary1[index] = np.nan
        print ('After\t', df.Salary1[index])
    index +=1

In [ ]:
for i, row in df.iterrows():a
    if row['Salary1'] > 20:            
        print (row['Salary1'])
    else:
        row['Salary1'] = np.nan

In [ ]:
df[['Salary1','Salary2']] = df['N_salary'].str.split('^\S*', expand=True)


In [ ]:
df.columns = ['year_quarter', 'unemployment_rate']
df['unemployment_rate'] = df['unemployment_rate'].map(lambda x: float(str(x).replace('%','')))
df.dropna(inplace=True)

In [ ]:
# Using replace and regex to clean the salary column

df[['salary']] = df[['salary']].replace('[Kk]\b', '000', regex=True)
df[['salary']] = df[['salary']].replace('\.[0-9][0-9] ', ' ', regex=True)
df[['salary']] = df[['salary']].replace('[!a-zA-Z+&,.\£/:)(]', '', regex=True)
df[['salary']] = df[['salary']].replace('....%', '', regex=True)

df.salary.str.strip().head()

In [ ]:
def remove_chars(s):
    return ''.join(re.findall(r'\d+[^-]+', s))

def remove_chars_another(data):
    return re.sub("[^0-9^.^-]", "", data)

def remove_dollar(data):
    return ''.join(re.findall(r'^\$?([0-9]{1,3},([0-9]{3},)*[0-9]{3}|[0-9]+)(.[0-9][0-9])?$', data))

\$(.*)  # grab everything after the first $ found
([^\s]+)  # grab fields except space - no spaces

^\S* # grab everything until first space found   
or ([^ ]+) .*

[^\s\s$][^\s\s$]{2,} just select the first numeric amount